In [1]:
import weightedcalcs as wc
import pandas as pd
import math
import numpy as np

In [2]:
cps = pd.read_csv('data/cps_00009.csv')

cps.drop(['SERIAL','HWTSUPP', 'CPSID', 'ASECFLAG','HFLAG','MONTH','PERNUM','CPSIDP'], 
         axis=1, 
         inplace=True)

workers = cps.query('INCWAGE != 9999999 and INCWAGE ! = 9999998 and INCWAGE > 0 and OCC2010 != 9920 and OCC2010 != 9999')

def industry_1990_group(row):
    ind = row['IND1990']
    if 10 <= ind <= 32 : return 1
    elif 40 <= ind <= 50 : return 2
    elif 60 <= ind <= 60 : return 3
    elif 100 <= ind <= 392 : return 4
    elif 400 <= ind <= 432 : return 5
    elif 440 <= ind <= 442 : return 6
    elif 450 <= ind <= 472 : return 7
    elif 500 <= ind <= 571 : return 8
    elif 580 <= ind <= 691 : return 9
    elif 700 <= ind <= 712 : return 10
    elif 721 <= ind <= 760 : return 11
    elif 761 <= ind <= 791 : return 12
    elif 800 <= ind <= 810 : return 13
    elif 812 <= ind <= 840 : return 14
    elif ind == 841 : return 15
    elif 842 <= ind <= 860 : return 16
    elif 861 <= ind <= 893 : return 17
    elif 900 <= ind <= 932 : return 18
    elif 940 <= ind <= 960 : return 19
    elif ind == 998 : return 0
    else: return 0
    
workers = workers.assign(IND_1990_GROUP = workers.apply(industry_1990_group, axis=1) )

In [3]:
def occ_2010_group(row):
    # https://cps.ipums.org/cps-action/variables/OCC2010#description_section
    occ = row['OCC2010']
    if 10 <= occ <= 430: return 1
    elif 500 <= occ <= 730: return 2
    elif 800 <= occ <= 950: return 3
    elif 1000 <= occ <= 1240: return 4
    elif 1300 <= occ <= 1540: return 5
    elif 1550 <= occ <= 1560: return 6
    elif 1600 <= occ <= 1980: return 7
    elif 2000 <= occ <= 2060: return 8
    elif 2100 <= occ <= 2150: return 9
    elif 2200 <= occ <= 2550: return 10
    elif 2600 <= occ <= 2920: return 11
    elif 3000 <= occ <= 3540: return 12
    elif 3600 <= occ <= 3650: return 13
    elif 3700 <= occ <= 3950: return 14
    elif 4000 <= occ <= 4150: return 15
    elif 4200 <= occ <= 4250: return 16
    elif 4300 <= occ <= 4650: return 17
    elif 4700 <= occ <= 4965: return 18
    elif 5000 <= occ <= 5940: return 19
    elif 6005 <= occ <= 6130: return 20
    elif 6200 <= occ <= 6765: return 21
    elif 6800 <= occ <= 6940: return 22
    elif 7000 <= occ <= 7630: return 23
    elif 7700 <= occ <= 8965: return 24
    elif 9000 <= occ <= 9750: return 25
    elif 9800 <= occ <= 9830: return 26
    elif 9920: return 0
    else: return 0
    
workers = workers.assign(OCC_2010_GROUP = workers.apply(occ_2010_group, axis=1)) 

In [4]:
def hispanic_origin(hispanic):
    if math.isnan(hispanic):
        return False
    elif 100 <= hispanic < 900:
        return True
    else: 
        return False

def race(row):
    hispanic = hispanic_origin( row['HISPAN'] )
    # non-hispanic white
    if row['RACE'] == 100 and not hispanic: return 'White'
    # non-hispanic black
    elif row['RACE'] == 200 and not hispanic: return 'Black'
    # non-hispanic asian or pacific islander
    elif (650 <= row['RACE'] <= 652) and not hispanic: return 'API'
    # # asian or pacific islander
    # elif (650 <= row['RACE'] <= 652): return 'API'
    # hispanic
    elif hispanic: return 'Latino'
    # multiracial/other
    else: return 'Other'

workers = workers.assign(RACE2 = workers.apply(race, axis=1))

In [5]:
def race_sex(row):
    if row['SEX'] == 1:
        sex = 'Men'
    elif row['SEX'] == 2:
        sex = 'Women'
    else:
        sex = 'None'
    return ' '.join([row['RACE2'], sex])
workers = workers.assign(RACE_SEX = workers.apply(race_sex, axis=1))

In [6]:
def immigrant(row):
    if math.isnan( row['CITIZEN']): return False
    elif 2 <= row['CITIZEN'] <= 3: return True
    else: return False
workers = workers.assign(IMMIGRANT = workers.apply(immigrant, axis=1))

In [7]:
def income_2015(row):
    income = row['INCWAGE']
    cpi = row['CPI99']
    income_1999 = income * cpi
    # https://cps.ipums.org/cps/cpi99.shtml
    return income_1999 * 1.430
workers = workers.assign(INCOME_2015 = workers.apply(income_2015, axis=1) )

In [8]:
def college(row):
    educ = row['EDUC']
    if educ == 999 or educ <= 73:
        # no college
        return 'N'
    else: 
        # some college or degrees
        return 'C'
    
workers = workers.assign(COLLEGE = workers.apply(college, axis=1) )    

In [9]:
calc = wc.Calculator('WTSUPP')

quintiles_over_time = pd.DataFrame()
for n in range(4):
    quintile = n + 1
    quintile_key = "quintile_%s" % (quintile)
    quintiles_over_time[quintile_key] = calc.quantile(workers.groupby('YEAR'), 'INCOME_2015', 0.2 * quintile)

# assign quintiles
def quintile(row):
    year = row['YEAR']
    income = row['INCOME_2015']
    q1 = quintiles_over_time['quintile_1'][year]
    q2 = quintiles_over_time['quintile_2'][year]
    q3 = quintiles_over_time['quintile_3'][year]
    q4 = quintiles_over_time['quintile_4'][year]
    
    if 0 < income <= q1:
        return 1
    elif q1 < income <= q2:
        return 2
    elif q2 < income <= q3:
        return 3
    elif q3 < income <= q4:
        return 4
    elif income > q4:
        return 5
    else:
        return 0
    
workers = workers.assign(QUINTILE = workers.apply(quintile, axis=1) )

In [10]:
quintiles_over_time.to_csv('csv/quintiles_over_time.csv')

In [11]:
deciles_over_time = pd.DataFrame()
for n in range(9):
    decile = n + 1
    decile_key = "decile_%s" % (decile)
    deciles_over_time[decile_key] = calc.quantile(workers.groupby('YEAR'), 'INCOME_2015', 0.1 * decile)

In [12]:
# assign deciles
def decile(row):
    year = row['YEAR']
    income = row['INCOME_2015']
    q1 = deciles_over_time['decile_1'][year]
    q2 = deciles_over_time['decile_2'][year]
    q3 = deciles_over_time['decile_3'][year]
    q4 = deciles_over_time['decile_4'][year]
    q5 = deciles_over_time['decile_5'][year]
    q6 = deciles_over_time['decile_6'][year]
    q7 = deciles_over_time['decile_7'][year]
    q8 = deciles_over_time['decile_8'][year]
    q9 = deciles_over_time['decile_9'][year]
    
    if 0 < income <= q1:
        return 1
    elif q1 < income <= q2:
        return 2
    elif q2 < income <= q3:
        return 3
    elif q3 < income <= q4:
        return 4
    elif q4 < income <= q5:
        return 5
    elif q5 < income <= q6:
        return 6
    elif q6 < income <= q7:
        return 7
    elif q7 < income <= q8:
        return 8
    elif q8 < income <= q9:
        return 9
    elif income > q9:
        return 10
    else:
        return 0
    
workers = workers.assign(DECILE = workers.apply(decile, axis=1) )

In [13]:
deciles_over_time.to_csv('csv/deciles_over_time.csv')

In [14]:
workers.to_csv('csv/workers.csv', index=False)